# Python Crash Course - Live Coding Session - with Solutions

# Setup

In [ ]:
# Imports go here
import os # operating-system level commands
import zipfile # unzipping files to folders
import numpy as np # Numpy is used for all types of array calculations

import cv2 # OpenCV (Open Computer Vision) for python. Helps with image manipulation
import matplotlib.pyplot as plt # Used for creating graphs with numbers
from google.colab.patches import cv2_imshow # Fixing a Colab issue here: https://stackoverflow.com/questions/65827830/disabledfunctionerror-cv2-imshow-is-disabled-in-colab-because-it-causes-jupy

!pip install --quiet binance-connector # Installing binance-connector, a package that is not preloaded on Colab
from binance.spot import Spot 

In [ ]:
# Useful notes about this environment:
# Our working directory is "/content". We can check our working directory by executing
print(os.getcwd())

In [ ]:
# Function to unzip a folder
def unzip(filename, foldername):
    with zipfile.ZipFile(filename, 'r') as file:
        file.extractall(foldername)

In [ ]:
# unzipping a folder
unzip("images.zip", "images")

# Create your own functions

In [ ]:
# MANIPULATING IMAGE
# -> blur image
# -> darken image

def manipulate_image(image, blur_strength, darken_strength):
    # Blurring:
    # https://www.geeksforgeeks.org/opencv-python-program-to-blur-an-image/
    # the cv2.blur() command takes two parameters, the image and the kernel size (usually equally distributed)
    img_manipulated = cv2.blur(image, (blur_strength,blur_strength))

    # Darkening image:
    # This is a super basic array transformation. Don't make things too complicated
    # https://stackoverflow.com/a/33333692
    img_manipulated = img_manipulated * (1-darken_strength)

    return img_manipulated


In [ ]:
# Crop IMAGE TO 16:9 format

def crop_portrait(image):
    height = image.shape[0]
    width = int(image.shape[0]/16*9)
    top = 0
    left = int(image.shape[1]/2 - width/2)
    return image[top:top+height, left:left+width]

In [ ]:
# Resizing an image to the desired height and width

def resize(img, height, width):
    # https://www.google.com/search?q=opencv+resize
    # https://www.tutorialkart.com/opencv/python/opencv-python-resize-image/
    result = cv2.resize(img, (height,width))
    return result

In [ ]:
# Paste one image on top of another one
def paste(img0, img1, top, left):
    # https://www.google.com/search?q=opencv+paste+image
    # https://stackoverflow.com/questions/14063070/overlay-a-smaller-image-on-a-larger-image-python-opencv
    result = img0.copy()
    result[top:top+img1.shape[0], left:left+img1.shape[1]] = img1
    return result

In [ ]:
# Adding text to image
# https://www.google.com/search?q=opencv+python+add+text+to+image
# https://stackoverflow.com/questions/16615662/how-to-write-text-on-a-image-in-windows-using-python-opencv2

def add_text(image, text):
    font                   = cv2.FONT_HERSHEY_SIMPLEX
    bottomLeftCornerOfText = (15,600)
    fontScale              = 1.5
    fontColor              = (255,255,255)
    thickness              = 4
    lineType               = 2

    result = cv2.putText(image,text, 
        bottomLeftCornerOfText, 
        font, 
        fontScale,
        fontColor,
        thickness,
        lineType)
    
    return result

In [ ]:
#Definition of a function: beware this code is not executed until you call the function!
def retrieve_BTC_data():
  """
  This function will return the close prices of the last hour according to https://accounts.binance.com as a list 
  return value: prices
  return datatype: List 
  """
  
  # This is the documentation of the API: https://binance-docs.github.io/apidocs/spot/en/#public-api-definitions 
  # Here are some examples https://github.com/binance/binance-connector-python

  # 1. Step: Make a connection to the website by using their API
  client = Spot()
  
  # 2. Step: Retrieve the close prices of the last hour using the function klines
  # This function returns klines which look like described in: https://binance-docs.github.io/apidocs/spot/en/#kline-candlestick-data
  klines_last_hour = client.klines("BTCUSDT", "1m", limit=61)
  print(klines_last_hour)

  # 3. Step: Filter out the essential data
  prices = []
  # Since we only want to retrieve the close price we do have to access the 5th element
  # within each element of the superior list. The easiest solution in this case and for our skill level is a loop.
  for i in klines_last_hour:
    prices.append(float(i[4]))
  #print(prices)
  return prices

In [ ]:
def plot_stonks(prices, x_lab, y_lab, pic_name):
  """
  This function will create a plot and also sace it as an .png
  prices: List
  x_lab: String Description of the x label
  y_lab: String Description of the x label
  pic_name: String Name of the image file
  """

  # 1. Step: pass the list of prices 
  plt.plot(prices)
  # 2. Step invert x-axis (otherwise it's confusing):
  # https://stackoverflow.com/questions/2051744/how-to-invert-the-x-or-y-axis
  plt.gca().invert_xaxis()
  # 3. Step (optional): describe the axes
  plt.ylabel(y_lab)
  plt.xlabel(x_lab)
  # 4. Step: Save the created plot!
  # We will create the name of the image file according to the passed name  
  plt.savefig(f"{pic_name}")
  # 5. Step: Show the created plot!
  plt.show()
  plt.close()

# Main Code

In [ ]:
# Stonks
# Now that we have our functions ready we can **call** these **functions**, store the returned values in a variable and plot the data.
# Retrieving bitcoin prices now works like this:
my_prices = retrieve_BTC_data()
# Plotting now works like this (we don't have to save the output here because we are not returning anything):
plot_stonks(my_prices, 'minutes', 'close price', 'BTCUSDT.png')
# Reading the image we just saved works like this:
stonk_img = cv2.imread("BTCUSDT.png")

# Looping through all images
# https://www.google.com/search?q=python+list+files+in+directory
# https://www.geeksforgeeks.org/python-list-files-in-a-directory/
images = os.listdir('images')
for filename in images:
    print(f"Editing image {filename}")
    # READING AN IMAGE
    # First we need to read an image
    # https://www.google.com/search?q=opencv+read+image
    # https://stackoverflow.com/questions/46540831/how-to-read-an-image-in-python-opencv
    # Solution (one line):
    img = cv2.imread(f'images/{filename}')
    img = crop_portrait(img)
    img = manipulate_image(img, 8, 0.4)

    # Remember to access the last element of a list you just have to use the index -1
    if my_prices[-1] < my_prices[0]:
        final = add_text(img, "STONKS! #YEY")
        final = paste(img, stonk_img, 100, 10)
    else:
        final = add_text(img, "NO STONKS! #SAD")
        final = paste(img, stonk_img, 100, 10)

    # Save image
    # https://www.google.com/search?q=python+opencv+save+image
    # https://www.geeksforgeeks.org/python-opencv-cv2-imwrite-method/
    cv2.imwrite(f"{filename}_edited.png", final)


In [ ]:
cv2_imshow(final)